## Hyperspy notebook to analyze Edx data denoised using XXX algorithm

#### by Martial Duchamp
NTU-MSE, Singapore m.duchamp@ntu.edu.sg

### Absract:
The notebook demonstrates usage of the XXX denoising algortyhm on EDX data
Results published in Nature XXX by Mevenkamp et al. 2017

### Preamble:

In [1]:
import hyperspy.api as hs

In [2]:
hs.hyperspy.Release.info

'\n    H y p e r S p y\n    Version 1.5.2\n\n    http://www.hyperspy.org\n\n    '

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%matplotlib qt4

#### Local functions

### Loading data:

In [7]:
nbr_xray_lines = 7

#### Raw data

In [ ]:
edx_raw = hs.load("./Raw_data/001_bot_layer.bcf", select_type='spectrum')

In [ ]:
edx_raw.plot()

In [ ]:
edx_raw.crop_signal1D(0.14792041,15.0) 

In [10]:
haadf_raw = hs.load("./Raw_data/HAADF_HAADF.png")

In [11]:
edx_raw.axes_manager

<Axes manager, axes: (744, 561|1486)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
           width |    744 |      0 |       0 | 1.2e-05 |     µm 
          height |    561 |      0 |       0 | 1.2e-05 |     µm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   1486 |        |    0.15 |    0.01 |    keV

In [12]:
haadf_raw_txt = np.loadtxt("./Raw_data/HAADF_HAADF.txt", dtype="float32", delimiter=';')

In [13]:
haadf_raw.data = haadf_raw_txt

In [14]:
haadf_raw.axes_manager[0].scale = edx_raw.axes_manager[0].scale
haadf_raw.axes_manager[0].units = edx_raw.axes_manager[0].units
haadf_raw.axes_manager[0].name = edx_raw.axes_manager[0].name
haadf_raw.axes_manager[1].scale = edx_raw.axes_manager[1].scale
haadf_raw.axes_manager[1].units = edx_raw.axes_manager[1].units
haadf_raw.axes_manager[1].name = edx_raw.axes_manager[1].name

edx_raw.set_signal_type('EDS_TEM')
edx_raw.set_microscope_parameters(beam_energy=200, live_time=2008, tilt_stage=0, azimuth_angle=45, elevation_angle=18, energy_resolution_MnKa=131.79)
edx_raw.change_dtype('float32')
edx_raw.metadata.Sample.xray_lines =['C_Ka', 'O_Ka', 'Al_Ka','Ga_La','Ga_Ka', 'In_La', 'N_Ka']

In [16]:
haadf_raw.axes_manager

Signal axis name,size,offset,scale,units
width,744,0.0,1.21249377472971e-05,µm
height,561,0.0,1.21249377472971e-05,µm


#### Denoised data

edx_denoised = hs.load("./Denoised_data/edx_highRes_lowSNR_denoised.hdf5")

haadf_denoised = hs.load("./Denoised_data/haadf_highRes_highSNR_denoised.tif")

haadf_denoised.axes_manager[0].scale = edx_denoised.axes_manager[0].scale
haadf_denoised.axes_manager[0].units = edx_denoised.axes_manager[0].units
haadf_denoised.axes_manager[0].name = edx_denoised.axes_manager[0].name
haadf_denoised.axes_manager[1].scale = edx_denoised.axes_manager[1].scale
haadf_denoised.axes_manager[1].units = edx_denoised.axes_manager[1].units
haadf_denoised.axes_manager[1].name = edx_denoised.axes_manager[1].name

edx_denoised.set_signal_type('EDS_TEM')
edx_denoised.set_microscope_parameters(beam_energy=200, live_time=2008, tilt_stage=0, azimuth_angle=45, elevation_angle=18, energy_resolution_MnKa=131.79)
edx_denoised.change_dtype('float32')
edx_denoised.metadata.Sample.xray_lines =['C_Ka', 'O_Ka', 'Al_Ka','Ga_La','Ga_Ka', 'In_La', 'N_Ka']

edx_denoised.plot(xray_lines=True)

haadf_denoised.axes_manager

# Field of view x
haadf_denoised.axes_manager[0].scale*haadf_denoised.axes_manager[0].size

# Field of view y
haadf_denoised.axes_manager[1].scale*haadf_denoised.axes_manager[1].size

### Plotting raw data

edx_raw.plot(navigator=haadf_raw, xray_lines=True)

for i in range(0,nbr_xray_lines):
    edx_raw.get_lines_intensity()[i].plot()

##### Mozaic Figure

fig, axes = plt.subplots(nrows=3, ncols=2, sharex=False, sharey=False, squeeze=True)
fig.subplots_adjust(hspace=-2.4, wspace=-1.0)
fig.set_tight_layout(True)
plt.setp(axes, xticks=[], xticklabels=[], yticks=[], yticklabels=[])

im = axes[0,0].imshow(haadf_raw_txt.data, cmap='gray')
im = axes[1,0].imshow(edx_raw.get_lines_intensity(['N_Ka'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[2,0].imshow(edx_raw.get_lines_intensity(['O_Ka'])[0].data, vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[0,1].imshow(edx_raw.get_lines_intensity(['Ga_La'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[1,1].imshow(edx_raw.get_lines_intensity(['Al_Ka'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[2,1].imshow(edx_raw.get_lines_intensity(['In_La'])[0].data, vmin = 0, vmax = 1, cmap='gnuplot')

plt.show()
plt.savefig("Raw_edx_fig.png", dpi=150)

### Plotting denoised data

edx_denoised.integrate1D(1).plot(navigator=haadf_denoised.integrate1D(1), xray_lines=True)

for i in range(0,nbr_xray_lines):
    edx_denoised.get_lines_intensity()[i].plot()

##### Mozaic Figure

fig, axes = plt.subplots(nrows=3, ncols=2, sharex=False, sharey=False, squeeze=True)
fig.subplots_adjust(hspace=-2.4, wspace=-1.0)
fig.set_tight_layout(True)
plt.setp(axes, xticks=[], xticklabels=[], yticks=[], yticklabels=[])

im = axes[0,0].imshow(haadf_denoised.data, cmap='gray')
im = axes[1,0].imshow(edx_denoised.get_lines_intensity(['N_Ka'])[0].data,vmin = 0, cmap='gnuplot')
im = axes[2,0].imshow(edx_denoised.get_lines_intensity(['O_Ka'])[0].data, vmin = 0, cmap='gnuplot')
im = axes[0,1].imshow(edx_denoised.get_lines_intensity(['Ga_La'])[0].data,vmin = 0, cmap='gnuplot')
im = axes[1,1].imshow(edx_denoised.get_lines_intensity(['Al_Ka'])[0].data,vmin = 0, cmap='gnuplot')
im = axes[2,1].imshow(edx_denoised.get_lines_intensity(['In_La'])[0].data, vmin = 0, cmap='gnuplot')

plt.show()
plt.savefig("Denoised_data_edx_fig2.png", dpi=150)

### Vertical integration of the integrated xray line, raw data

In [ ]:
Import integrated data from AZtec

In [ ]:
Int_edx_data = np.loadtxt("./Raw_data/EDX.csv", dtype="float32", delimiter=';') #numpy array
#2D x: nm, y:at%

In [ ]:
#Create Hyperspy object
Int_edx_data_hyp = hs.signals.Signal2D(Int_edx_data ...)

In [31]:
Int_ver = []

Int_ver.append(norm(haadf_raw.integrate1D(1)))
Int_ver.append(Int_edx_data_hyp[0]) #Pb
Int_ver.append(norm(edx_raw.get_lines_intensity(['Ga_La'])[0].integrate1D(1)).transpose())
Int_ver.append(norm(edx_raw.get_lines_intensity(['Al_Ka'])[0].integrate1D(1)).transpose())
Int_ver.append(norm(edx_raw.get_lines_intensity(['In_La'])[0].integrate1D(1)).transpose())

In [32]:
#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Int_ver, style='cascade', legend=None, dpi=200)
cascade_plot.figure.savefig("Vertical_integration_raw_data.png")


In [33]:
#### Overlap plot

cascade_plot = hs.plot.plot_spectra(Int_ver, style='overlap', legend='auto', legend_loc='center left')
cascade_plot.figure.savefig("Vertical_integration.png")

### Vertical integration of the integrated xray line, denoised data

Int_ver = []

Int_ver.append(norm(haadf_denoised.integrate1D(1)))
Int_ver.append(norm(edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_denoised.get_lines_intensity(['Ga_La'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose()))

Int_ver = []

Int_ver.append(norm(haadf_denoised.integrate1D(1)))
Int_ver.append((edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose())/1.526)
Int_ver.append((edx_denoised.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose())/2.18)
Int_ver.append((edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose())/0.993)
Int_ver.append((edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose())/3.071)

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Int_ver, style='cascade', legend=None, legend_loc='center left', dpi=200)
cascade_plot.figure.savefig("Vertical_integration_denoised_data.png")


#### Overlap plot

cascade_plot = hs.plot.plot_spectra(Int_ver, style='overlap', legend=None, legend_loc='center left')
cascade_plot.figure.savefig("Vertical_integration.png")

### Quantification of vertically integrated raw data

# CL factor from Rosenmoheur paper at 300kV (from Esprit)
# CL factor from Esprit software juelich (see text file)

CL_Ga_Ka=2.18 #Ga K: 2.18 Ga L:2.006
CL_Al_Ka=0.993 #1.2
CL_In_La=3.071 #2.9 #3.05 #In K: 41.184 In L:3.071
CL_N_K=1.526

edx_raw.get_lines_intensity()

edx_raw.get_lines_intensity()

Sum_Ga_Al_In_raw = mul_CL(edx_raw.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka)+mul_CL(edx_raw.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(), CL_Al_Ka)+mul_CL(edx_raw.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(), CL_In_La)+mul_CL(edx_raw.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(), CL_N_K)

Int_ver = []

Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka)/Sum_Ga_Al_In_raw)
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(),CL_Al_Ka)/Sum_Ga_Al_In_raw)
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(),CL_In_La)/Sum_Ga_Al_In_raw)
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(),CL_N_K)/Sum_Ga_Al_In_raw)

Int_ver = []

Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka))
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(),CL_Al_Ka))
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(),CL_In_La))
Int_ver.append(mul_CL(edx_raw.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(),CL_N_K))

Ga_Ka_raw = edx_raw.integrate_in_range(signal_range=(9.0,9.5))/(9.5-9.0)
#Ga_La = edx_denoised.integrate_in_range(signal_range=(1.0,1.23))/(1.23-1.0)
N_Ka_raw = edx_raw.integrate_in_range(signal_range=(0.34,0.44))/(0.44-0.34)
In_La_raw = edx_raw.integrate_in_range(signal_range=(3.15,3.4))/(3.4-3.15)
Al_Ka_raw = edx_raw.integrate_in_range(signal_range=(1.35,1.65))/(1.65-1.35)

Int_ver = []

Int_ver.append(Ga_Ka_raw.integrate1D(1).transpose())
Int_ver.append(Al_Ka_raw.integrate1D(1).transpose())
Int_ver.append(In_La_raw.integrate1D(1).transpose())
Int_ver.append(N_Ka_raw.integrate1D(1).transpose())

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Int_ver, style='overlap', dpi=200)
cascade_plot.figure.savefig("Vertical_integration_raw_data_not-norm_not-CL_overlap.png")


### Quantification of vertically integrated denoised data

edx_denoised.plot(xray_lines=True)

# CL factor from Rosenmoheur paper at 300kV (from Esprit)
# CL factor from Esprit software juelich (see text file)

CL_Ga_Ka=2.18 #Ga K: 2.18 Ga L:2.006
CL_Ga_La=2.006
CL_Al_Ka=0.993 #1.2
CL_In_La=3.071 #2.9 #3.05 #In K: 41.184 In L:3.071
CL_N_K=1.526

Sum_Ga_Al_In = mul_CL(edx_denoised.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka)+mul_CL(edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(), CL_Al_Ka)+mul_CL(edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(), CL_In_La)+mul_CL(edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(), CL_N_K)

Int_ver = []

Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka)/Sum_Ga_Al_In)
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(),CL_Al_Ka)/Sum_Ga_Al_In)
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(),CL_In_La)/Sum_Ga_Al_In)
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(),CL_N_K)/Sum_Ga_Al_In)

Int_ver = []

Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose(),CL_Ga_Ka))
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose(),CL_Al_Ka))
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose(),CL_In_La))
Int_ver.append(mul_CL(edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose(),CL_N_K))

Int_ver = []

Int_ver.append((edx_denoised.get_lines_intensity(['Ga_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append((edx_denoised.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append((edx_denoised.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose()))
Int_ver.append((edx_denoised.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose()))

Ga_Ka_denoised = edx_denoised.integrate_in_range(signal_range=(9.0,9.5))/(9.5-9.0)
#Ga_La = edx_denoised.integrate_in_range(signal_range=(1.0,1.23))/(1.23-1.0)
N_Ka_denoised = edx_denoised.integrate_in_range(signal_range=(0.34,0.44))/(0.44-0.34)
In_La_denoised = edx_denoised.integrate_in_range(signal_range=(3.15,3.4))/(3.4-3.15)
Al_Ka_denoised = edx_denoised.integrate_in_range(signal_range=(1.35,1.65))/(1.65-1.35)

Int_ver = []

Int_ver.append(Ga_Ka_denoised.integrate1D(1).transpose())
Int_ver.append(Al_Ka_denoised.integrate1D(1).transpose())
Int_ver.append(In_La_denoised.integrate1D(1).transpose())
Int_ver.append(N_Ka_denoised.integrate1D(1).transpose())

Int_ver = []

Int_ver.append(mul_CL(Ga_Ka.integrate1D(1).transpose(),CL_Ga_Ka))
Int_ver.append(mul_CL(Ga_La.integrate1D(1).transpose(),CL_Ga_La))
Int_ver.append(mul_CL(Al_Ka.integrate1D(1).transpose(),CL_Al_Ka))
Int_ver.append(mul_CL(In_La.integrate1D(1).transpose(),CL_In_La))
Int_ver.append(mul_CL(N_Ka.integrate1D(1).transpose(),CL_N_K))

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Int_ver, style='overlap', dpi=200)
cascade_plot.figure.savefig("Vertical_integration_denoised_data_CL_overlap.png")

### Maximun and average Spectrum

local_spc_a = edx_raw.mean(axis=1).mean(axis=0).deepcopy()

local_spc_b = local_spc_1.deepcopy()

local_spc_a.data = edx_raw.data[280,372,:]/20
local_spc_b.data = edx_denoised.data[280,372,:]

Spc_lt = []

#Middle Spectra
Spc_lt.append(local_spc_a)
Spc_lt.append(local_spc_b)
#Average Spectra
Spc_lt.append(edx_raw.mean(axis=1).mean(axis=0))
Spc_lt.append(edx_denoised.mean(axis=1).mean(axis=0))

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Spc_lt, style='cascade', legend=None, legend_loc='center left', dpi=200)
cascade_plot.figure.savefig("Spectra_extracted.png")

### Average in GaN layer to detect the presence of Al

edx_raw.plot()

edx_raw.data = np.ma.array(edx_raw.data, mask=False)

edx_raw.data.mask[:,100:] = True

edx_raw.plot()

Spc_lt = []

#Average Spectra
Spc_lt.append(edx_raw.mean(axis=1).mean(axis=0))

edx_raw.data.mask[:,:] = False

edx_raw.data.mask[:,200:] = True

edx_raw.plot()

Spc_lt.append(edx_raw.mean(axis=1).mean(axis=0))

edx_raw.data.mask[:,:] = False

edx_raw.plot()

Spc_lt.append(edx_raw.mean(axis=1).mean(axis=0))

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Spc_lt, style='cascade', legend=None, dpi=200)
cascade_plot.figure.savefig("Spectra_extracted_x.png")

### PCA Denoising

edx_raw.change_dtype('float64')

edx_raw.decomposition(normalize_poissonian_noise=True, output_dimension=3, navigation_mask=None)

edx_raw.plot_explained_variance_ratio()

edx_raw_sc = edx_raw.get_decomposition_model(components=10)

edx_raw_sc.save("PCA_10compo.hdf5")

edx_raw_sc.plot()

edx_raw_sc.mean(axis=1).mean(axis=0).plot()

### Mozaic figure

fig, axes = plt.subplots(nrows=3, ncols=2, sharex=False, sharey=False, squeeze=True)
fig.subplots_adjust(hspace=-2.4, wspace=-1.0)
fig.set_tight_layout(True)
plt.setp(axes, xticks=[], xticklabels=[], yticks=[], yticklabels=[])

im = axes[0,0].imshow(haadf_raw.data, cmap='gray')
im = axes[1,0].imshow(edx_raw_sc.get_lines_intensity(['N_Ka'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[2,0].imshow(edx_raw_sc.get_lines_intensity(['Ga_La'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[0,1].imshow(edx_raw_sc.get_lines_intensity(['Al_Ka'])[0].data,vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[1,1].imshow(edx_raw_sc.get_lines_intensity(['In_La'])[0].data, vmin = 0, vmax = 1, cmap='gnuplot')
im = axes[2,1].imshow(edx_raw_sc.get_lines_intensity(['In_La'])[0].data, vmin = 0, vmax = 1, cmap='gnuplot')


plt.show()
plt.savefig("Denoised_data_edx_fig2_PCA10.png", dpi=150)

Int_ver = []

Int_ver.append(norm(haadf_raw.integrate1D(1)))
Int_ver.append(norm(edx_raw_sc.get_lines_intensity(['N_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_raw_sc.get_lines_intensity(['Ga_La'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_raw_sc.get_lines_intensity(['Al_Ka'])[0].integrate1D(1).transpose()))
Int_ver.append(norm(edx_raw_sc.get_lines_intensity(['In_La'])[0].integrate1D(1).transpose()))

#### Cascade plot
cascade_plot = hs.plot.plot_spectra(Int_ver, style='cascade', legend=None, legend_loc='center left', dpi=200)
cascade_plot.figure.savefig("Vertical_integration_raw_PCA10.png")


haadf_001 = hs.load("./Raw_data/emi/001_1.8Mx_15.00.02 Scanning Acquire_1.ser")

haadf_001.plot()

haadf_002 = hs.load("./Raw_data/emi/001_7.2Mx_15.01.20 Scanning Acquire_1.ser")

haadf_002.plot()

haadf_003 = hs.load("./Raw_data/emi/001_7.2Mx__rot90deg_15.01.20 Scanning Acquire_1.ser")

haadf_003.plot()

haadf_004 = hs.load("./Raw_data/emi/002_15.17.03 Scanning Acquire_1.ser")

haadf_004.plot()

haadf_005 = hs.load("./Raw_data/emi/004_3rd_layer_15.44.52 Scanning Acquire_1.ser")

haadf_005.plot()

edx_raw.mean([0,1,2]).data